# Experiment 05: Call-Tree Profiling

## 1. Rationale

**Goal:** Identify hot paths in RadiObject's I/O stack using pyinstrument call-tree profiling.

Each section profiles a key operation and renders an interactive HTML flamegraph inline.

In [ ]:
# Parameters (papermill)
BATCH_SIZE = 4
PATCH_SIZE = (64, 64, 64)
NUM_WORKERS = 0
N_WARMUP = 5
N_RUNS = 10
N_BATCHES = 20
N_SUBJECTS = 20
RANDOM_SEED = 42
# S3_BUCKET = "souzy-scratch"
TILING_STRATEGIES = ["axial", "isotropic"]

In [ ]:
import sys
import tempfile
from pathlib import Path

from IPython.display import HTML, display
from pyinstrument import Profiler

# Derive project root from absolute config paths
from benchmarks.config import _BENCHMARKS_DIR, BENCHMARK_DIR, NIFTI_DIR, S3_BUCKET, S3_REGION

project_root = _BENCHMARKS_DIR.parent
sys.path.insert(0, str(project_root / "src"))

from radiobject import RadiObject
from radiobject.ctx import S3Config, configure

In [ ]:
# Datasets
nifti_paths = sorted(NIFTI_DIR.glob("*.nii.gz"))[:5]
assert nifti_paths, f"No NIfTI files found in {NIFTI_DIR}"

radi_axial = RadiObject(str(BENCHMARK_DIR / "radiobject-axial"))
radi_isotropic = RadiObject(str(BENCHMARK_DIR / "radiobject-isotropic"))

configure(s3=S3Config(region=S3_REGION))
radi_s3 = RadiObject(f"s3://{S3_BUCKET}/benchmark/radiobject-axial")

print(f"NIfTI files: {len(nifti_paths)}")
print(f"Local axial: {len(radi_axial)} subjects")
print(f"S3 axial: {len(radi_s3)} subjects")

## 2. Profile: `from_images()` Ingestion

In [ ]:
profiler = Profiler()
profiler.start()

with tempfile.TemporaryDirectory() as tmpdir:
    RadiObject.from_images(
        uri=str(Path(tmpdir) / "profiled"),
        images={"image": str(NIFTI_DIR / "*.nii.gz")},
    )

profiler.stop()
display(HTML(profiler.output_html()))

## 3. Profile: `Volume.to_numpy()` Full Read

In [ ]:
vol = radi_axial.collection(list(radi_axial.collection_names)[0]).iloc[0]

profiler = Profiler()
profiler.start()

for _ in range(N_RUNS):
    _ = vol.to_numpy()

profiler.stop()
display(HTML(profiler.output_html()))

## 4. Profile: `Volume.axial()` Partial Read

In [ ]:
vol = radi_axial.collection(list(radi_axial.collection_names)[0]).iloc[0]
mid_z = vol.shape[2] // 2

profiler = Profiler()
profiler.start()

for _ in range(100):
    _ = vol.axial(mid_z)

profiler.stop()
display(HTML(profiler.output_html()))

## 5. Profile: `Volume.slice()` 3D ROI Extraction

In [ ]:
vol = radi_isotropic.collection(list(radi_isotropic.collection_names)[0]).iloc[0]

profiler = Profiler()
profiler.start()

for _ in range(100):
    _ = vol.slice(x=slice(0, 64), y=slice(0, 64), z=slice(0, 64))

profiler.stop()
display(HTML(profiler.output_html()))

## 6. Profile: S3 Full Volume Read

In [ ]:
vol = radi_s3.collection(list(radi_s3.collection_names)[0]).iloc[0]

profiler = Profiler()
profiler.start()

for _ in range(5):
    _ = vol.to_numpy()

profiler.stop()
display(HTML(profiler.output_html()))

## 7. Key Findings

1. **Ingestion:** Dominated by NIfTI parsing (nibabel) and TileDB array creation
2. **Full Read:** TileDB multi-range query + numpy copy
3. **Partial Read:** Tile-aligned reads minimize I/O; misaligned reads show extra tile fetches
4. **S3:** Network latency dominates; TileDB VFS handles chunked transfer